<a href="https://colab.research.google.com/github/24p11/recode-scenario/blob/main/scenario_oncology_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Create fictive clinical notes from Code set (DRG + ICD)

Code set are the raw classification data, we can extract from National database (Base nationale PMSI en France). They are made of 
* classification profile made of grouping variables from DRG records which are prepared with their frequency in the national database
    - age (class)
    - sexe
    - DRG (racine GHM)
    - Main diagnosis (ICD10) : cf
    - Hospitalization management type : cf
* diagnosis associated to each classification profile, extracted with their frequencies
* procedures associated to each classification profile, specialy for surgery and technical gestures, extracted with their frequencies

From thoses raw information we produce a coded clinical scenario which will be uses a seed.

This scenario is transformed into a detail prompt that will be given to a LLM for generation.
From the combinaision of primary and related diagnosis in French discharge abstract, we derived two notions :
* Primary diagnosis : host the notion of principal pathology, it is rather the primary diagnosis of the discharge abstract or the related diagnosis when it exists and that the primary diagnosis of the discharge abstract is from the chapter "Facteurs influant sur l’état de santé" of ICD10
* The Hospitalization management type is rather the term "Primary diagnosis" or the ICD-10 code of the related diagnosis when it exists


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import datetime as dt

In [3]:
from utils import *

In [4]:
gs = generate_scenario()
# Load official dictionaries
# col_names option allow you to algin your column names the project dictionary.
gs.load_offical_icd("cim_2024.xlsx",col_names={"code" : "icd_code","libelle":"icd_code_description"} )
gs.load_offical_procedures("ccam_actes_2024.xlsx",col_names={"code":"procedure","libelle_long":"procedure_description"} )
col_names={"Code CIM":"icd_parent_code","Localisation":"primary_site","Type Histologique":"histological_type",
           "Stade":"stage","Marqueurs Tumoraux":"biomarkers","Traitement":"treatment_recommandation","Protocole de Chimiothérapie":"chemotherapy_regimen"}
gs.load_cancer_treatement_recommandations("Tableau récapitulatif traitement cancer.xlsx",col_names ) 
col_names={"racine":"drg_parent_code","lib_spe_uma":"specialty","ratio_spe_racine":"ratio"}
gs.load_specialty_refential("dictionnaire_spe_racine.xlsx",col_names)
gs.load_referential_hospital("chu")

In [5]:
# Load data from BN  PMSI
col_names={"racine":"drg_parent_code","das": "icd_secondary_code","diag":"icd_primary_code","categ_cim":"icd_primary_parent_code",
            "mdp":"case_management_type","nb_situations":"nb","acte":"procedure",
            "mode_entree":"admission_mode",
            "mode_sortie":"discharge_disposition",
            "mode_hospit":"admission_type"}
gs.load_classification_profile("bn_pmsi_cases_20250819.csv", col_names)
gs.load_secondary_icd("bn_pmsi_related_diag_20250818.csv",col_names)
gs.load_procedures("bn_pmsi_procedures_20250818.csv",col_names)

In [6]:
#Prepare cases
# df_profile = gs.df_classification_profile.drop(columns="nb")
df_profile = gs.df_classification_profile
df_profile = df_profile[(df_profile.icd_primary_code.isin(gs.icd_codes_cancer) )  & (~df_profile.drg_parent_code.isin(gs.drg_parent_code_radio) ) ]

In [7]:
def create_system_prompt(scenario):
    if scenario['admission_type'] == "Inpatient" and scenario['drg_parent_code'][2:3]=="C" :
        template_name = "surgery_complete.txt"
    elif scenario['admission_type'] == "Outpatient" and scenario['drg_parent_code'][2:3]=="C" :
        template_name = "surgery_outpatient.txt"
    else:
        template_name = "scenario_onco_v1.txt"
        
    with open("templates/" + template_name, "r", encoding="utf-8") as f:
        prompt = f.read()
    
    return prompt

In [8]:
# col_names = ["icd_primary_code", "case_management_type", "drg_parent_code", "cage2","cage", "sexe", "admission_type","admission_mode", "discharge_disposition",
#              "dms", "los_mean", "los_sd", "drg_parent_description"]
# test_generation_v1 = df_profile.sample(20)[col_names].reset_index(drop=True)
test_generation_v1 = df_profile.sample(1000, weights="nb").reset_index(drop=True)
test_generation_v1

,icd_primary_code,case_management_type,drg_parent_code,age,cage,cage2,sexe,admission_type,admission_mode,discharge_disposition,...,libelle_da,gp_cas,libelle_gp_cas,ga,libelle_ga,da_gp,da_gp_ga,anseqta,aso,specialty
0,C800,Z515,23Z02,ge_18,[80-[,[50-[,1,Inpatient,URGENCES,DECES,...,"Douleurs chroniques, Soins palliatifs",X22,Douleur et soins palliatifs,G176,Soins palliatifs,D24X22,D24X22G176,2024,M,SOINS PALLIATIFS
1,C67,Z512,28Z17,ge_18,[70-80[,[50-[,1,Outpatient,DOMICILE,DOMICILE,...,Séances,S03,Chimiothérapie pour aff. Non tumorale,G191,Séances : chimiothérapie non tumorale,D27S03,D27S03G191,2024,M,NaN
2,C795,DP,08M25,ge_18,[50-60[,[50-[,2,Outpatient,DOMICILE,DOMICILE,...,Rhumatologie,X06,Rhumatologie,G038,Prise en charge médicale des tumeurs de l'appa...,D04X06,D04X06G038,2024,M,RHUMATOLOGIE
3,C182,Z452,05K14,ge_18,[70-80[,[50-[,1,Outpatient,DOMICILE,DOMICILE,...,Cardio-vasculaire (hors cathétérismes vasculai...,K06,Mise en place d'accès vasculaire,G199,Mise en place d'accès vasculaire,D07K06,D07K06G199,2024,C,CHIRURGIE VISCERALE
4,C795,DP,08M25,ge_18,[80-[,[50-[,2,Outpatient,DOMICILE,DOMICILE,...,Rhumatologie,X06,Rhumatologie,G038,Prise en charge médicale des tumeurs de l'appa...,D04X06,D04X06G038,2024,M,RHUMATOLOGIE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,C50,DP,09C14,ge_18,[60-70[,[50-[,2,Inpatient,DOMICILE,DOMICILE,...,"Activités inter spécialités, suivi thérapeutiq...",C25,Chirurgie inter spécialités,G179,Chirurgie inter spécialités,D26C25,D26C25G179,2024,C,CHIRURGIE VISCERALE
996,C182,DP,06M05,ge_18,[80-[,[50-[,1,Inpatient,DOMICILE,DOMICILE,...,Digestif,X02,Hépato-Gastro-Entérologie,G011,Prise en charge médicale des tumeurs malignes ...,D01X02,D01X02G011,2024,M,HEPATO-GASTRO-ENTERO
997,D381,DP,04K02,ge_18,[50-60[,[50-[,2,Outpatient,DOMICILE,DOMICILE,...,Pneumologie,K07,"Endoscopies bronchiques, avec ou sans anesthésie",G084,"Endoscopies bronchiques, avec ou sans anesthésie",D09K07,D09K07G084,2024,M,PNEUMOLOGIE
998,C819,Z452,05K14,ge_18,[18-30[,[18-50[,1,Outpatient,DOMICILE,DOMICILE,...,Cardio-vasculaire (hors cathétérismes vasculai...,K06,Mise en place d'accès vasculaire,G199,Mise en place d'accès vasculaire,D07K06,D07K06G199,2024,C,CHIRURGIE VISCERALE


In [9]:
from tqdm import tqdm

In [10]:
list_scenario = []

for i in tqdm(range(len(test_generation_v1))):
# for i in tqdm(range(1)):
    profile = test_generation_v1.iloc[i].copy()
    scenario = gs.generate_scenario_from_profile(profile.drop("nb"))
    row = {k:scenario[k] for k in scenario.keys()}
    user_prompt = gs.make_prompts_marks_from_scenario(scenario)
    system_prompt = create_system_prompt(scenario)
    row["user_prompt"] = user_prompt
    row["system_prompt"] = system_prompt
    
    if  scenario["icd_primary_code"]  in gs.icd_codes_cancer  :
        prefix = """Le compte rendu suivant respecte les élements suivants :
        - les diagnostics ont une formulation moins formelle que la définition du code
        - le type histologique et la valeur des biomarqueurs si recherchés
        - le plan du CRH est conforme aux recommandations.
        """
    else :
        prefix = """Le compte rendu suivant respecte les élements suivants :
        - les diagnostics ont une formulation moins formelle que la définition du code
        - le plan du CRH est conforme aux recommandations.
        """
    
    row["prefix"] = prefix
    row["prefix_len"] = len(prefix)
    list_scenario.append(row)

100%|██████████| 1000/1000 [47:30<00:00,  2.85s/it]


In [11]:
keep_cols = ['age',  'cage', 'cage2','sexe', 'date_entry', 'date_discharge', 'date_of_birth',
       'first_name', 'last_name', 'icd_primary_code', 'icd_primary_description', 'icd_parent_code',
       'case_management_type','case_management_type_description', 'case_management_type_text', 
       'drg_parent_code', 'drg_parent_description',
       'icd_secondaray_code',  'text_secondary_icd_official', 
        'procedure', 'text_procedure',
        'admission_type','admission_mode', 'discharge_disposition', 'dms', 'los_mean', 'los_sd',
       'cancer_stage', 'score_TNM', 'histological_type',
       'treatment_recommandation', 'chemotherapy_regimen', 'biomarkers',
       'first_name_med', 'last_name_med',
       'department','hospital',
       'cd_md_pec', 'user_prompt', 'system_prompt', 'prefix', 'prefix_len']
df_scenario = pd.DataFrame(list_scenario)[keep_cols]
df_scenario

,age,cage,cage2,sexe,date_entry,date_discharge,date_of_birth,first_name,last_name,icd_primary_code,...,biomarkers,first_name_med,last_name_med,department,hospital,cd_md_pec,user_prompt,system_prompt,prefix,prefix_len
0,84,[80-[,[50-[,1,2024-05-14,2024-05-24,1940-03-05,Nicole,Rosset-boulon,C800,...,None,Katica,Runigo,None,Nouvel Hôpital Civil - Hôpitaux Universitaires...,8,**SCÉNARIO DE DÉPART :**\n- Âge du patient : 8...,Vous êtes un oncologue clinicien expert. Votre...,Le compte rendu suivant respecte les élements ...,286
1,77,[70-80[,[50-[,1,2024-08-15,2024-08-15,1947-02-10,Celestin,Picher,C67,...,None,Danielle,Schisler,None,Hôpital Michallon - CHU Grenoble Alpes,15,**SCÉNARIO DE DÉPART :**\n- Âge du patient : 7...,Vous êtes un oncologue clinicien expert. Votre...,Le compte rendu suivant respecte les élements ...,286
2,56,[50-60[,[50-[,2,2024-05-24,2024-05-24,1968-02-21,Aurelie,Bies dit lapoudje,C795,...,None,Younesse,Doudot,None,Hôpital de Hautepierre - Hôpitaux Universitair...,16,**SCÉNARIO DE DÉPART :**\n- Âge du patient : 5...,Vous êtes un oncologue clinicien expert. Votre...,Le compte rendu suivant respecte les élements ...,286
3,73,[70-80[,[50-[,1,2024-01-09,2024-01-09,1950-08-23,Jean,Maubert,C182,...,None,Jean,St leon,None,Centre Hospitalier Lyon Sud - Hospices Civils ...,13,**SCÉNARIO DE DÉPART :**\n- Âge du patient : 7...,Vous êtes un oncologue clinicien expert. Votre...,Le compte rendu suivant respecte les élements ...,286
4,80,[80-[,[50-[,2,2024-06-05,2024-06-05,1943-11-22,Yvette,Aubin,C795,...,None,Gaël,Basta,None,Hôpital Cochin - Assistance Publique Hôpitaux ...,16,**SCÉNARIO DE DÉPART :**\n- Âge du patient : 8...,Vous êtes un oncologue clinicien expert. Votre...,Le compte rendu suivant respecte les élements ...,286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,68,[60-70[,[50-[,2,2024-07-19,2024-07-20,1956-05-16,Madeleine,Portail,C50,...,"ER/PR, HER2",Petru,Minassian,None,Hôpital Saint-Louis - Assistance Publique Hôpi...,12,**SCÉNARIO DE DÉPART :**\n- Âge du patient : 6...,Vous êtes chirurgien praticien. Votre tâche es...,Le compte rendu suivant respecte les élements ...,286
996,88,[80-[,[50-[,1,2024-04-11,2024-04-14,1935-11-15,Dion,Zappa,C182,...,CEA,Anoir,Badoraly,None,Centre Hospitalier Lyon Sud - Hospices Civils ...,16,**SCÉNARIO DE DÉPART :**\n- Âge du patient : 8...,Vous êtes un oncologue clinicien expert. Votre...,Le compte rendu suivant respecte les élements ...,286
997,59,[50-60[,[50-[,2,2024-06-18,2024-06-18,1964-11-15,Khadidia,Marquot,D381,...,None,Marie-jeanne,Choulet,None,Hôpital Roger Salengro - Centre Hospitalier Un...,13,**SCÉNARIO DE DÉPART :**\n- Âge du patient : 5...,Vous êtes un oncologue clinicien expert. Votre...,Le compte rendu suivant respecte les élements ...,286
998,25,[18-30[,[18-50[,1,2024-10-21,2024-10-21,1999-08-17,Christian,Givron,C819,...,None,Bérengère,Arnott,None,Hôpital Pellegrin - CHU de Bordeaux,13,**SCÉNARIO DE DÉPART :**\n- Âge du patient : 2...,Vous êtes un oncologue clinicien expert. Votre...,Le compte rendu suivant respecte les élements ...,286


In [12]:
df_scenario.to_csv("data_1000.csv")